In [38]:
!pip install xgboost==1.5.1

In [3]:
import pandas as pd
import numpy as np 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost import XGBClassifier
import xgboost as xgb

In [4]:
df = pd.read_parquet('updated_fitness.parquet')

In [5]:
df

,Date,total_sugar,total_sodium,total_carbs,total_protein,total_fat,total_calories,goal_sugar,goal_sodium,goal_carbs,goal_protein,goal_fat,goal_calories,sugar_difference,sodium_difference,carbs_difference,protein_difference,fat_difference,calories_difference,Run Decision
0,2014-09-14,109.0,3658.0,340.0,186.0,114.0,2924.0,119.0,2300.0,396.0,160.0,105,3173.0,10.0,-1358.0,56.0,-26.0,-9.0,249.0,No
1,2014-09-15,63.0,855.0,96.0,50.0,37.0,2430.0,59.0,2300.0,196.0,79.0,52,1572.0,-4.0,1445.0,100.0,29.0,15.0,-858.0,No
2,2014-09-16,100.0,2215.0,158.0,114.0,54.0,1862.0,69.0,2300.0,229.0,92.0,61,1832.0,-31.0,85.0,71.0,-22.0,7.0,-30.0,No
3,2014-09-17,105.0,1765.0,187.0,98.0,60.0,2251.0,63.0,2300.0,210.0,85.0,56,1685.0,-42.0,535.0,23.0,-13.0,-4.0,-566.0,No
4,2014-09-18,71.0,1101.0,113.0,202.0,81.0,2001.0,60.0,2300.0,199.0,80.0,53,1597.0,-11.0,1199.0,86.0,-122.0,-28.0,-404.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587182,2015-03-02,0.0,0.0,0.0,0.0,0.0,0.0,76.0,0.0,252.0,101.0,67,2017.0,76.0,0.0,252.0,101.0,67.0,2017.0,Yes
587183,2015-03-03,0.0,0.0,0.0,0.0,0.0,0.0,67.0,0.0,225.0,90.0,60,1799.0,67.0,0.0,225.0,90.0,60.0,1799.0,Yes
587184,2015-03-04,0.0,0.0,0.0,0.0,0.0,0.0,72.0,0.0,240.0,96.0,64,1920.0,72.0,0.0,240.0,96.0,64.0,1920.0,Yes
587185,2015-03-14,0.0,0.0,0.0,0.0,0.0,0.0,79.0,0.0,265.0,106.0,71,2118.0,79.0,0.0,265.0,106.0,71.0,2118.0,Yes


In [6]:
df['Date']=pd.to_datetime(df['Date'])

In [7]:
df['Day']=pd.to_datetime(df['Date']).dt.day

In [8]:
df['Month']=pd.to_datetime(df['Date']).dt.month

In [9]:
df['Year']=pd.to_datetime(df['Date']).dt.year

In [10]:
df

,Date,total_sugar,total_sodium,total_carbs,total_protein,total_fat,total_calories,goal_sugar,goal_sodium,goal_carbs,...,sugar_difference,sodium_difference,carbs_difference,protein_difference,fat_difference,calories_difference,Run Decision,Day,Month,Year
0,2014-09-14,109.0,3658.0,340.0,186.0,114.0,2924.0,119.0,2300.0,396.0,...,10.0,-1358.0,56.0,-26.0,-9.0,249.0,No,14,9,2014
1,2014-09-15,63.0,855.0,96.0,50.0,37.0,2430.0,59.0,2300.0,196.0,...,-4.0,1445.0,100.0,29.0,15.0,-858.0,No,15,9,2014
2,2014-09-16,100.0,2215.0,158.0,114.0,54.0,1862.0,69.0,2300.0,229.0,...,-31.0,85.0,71.0,-22.0,7.0,-30.0,No,16,9,2014
3,2014-09-17,105.0,1765.0,187.0,98.0,60.0,2251.0,63.0,2300.0,210.0,...,-42.0,535.0,23.0,-13.0,-4.0,-566.0,No,17,9,2014
4,2014-09-18,71.0,1101.0,113.0,202.0,81.0,2001.0,60.0,2300.0,199.0,...,-11.0,1199.0,86.0,-122.0,-28.0,-404.0,No,18,9,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587182,2015-03-02,0.0,0.0,0.0,0.0,0.0,0.0,76.0,0.0,252.0,...,76.0,0.0,252.0,101.0,67.0,2017.0,Yes,2,3,2015
587183,2015-03-03,0.0,0.0,0.0,0.0,0.0,0.0,67.0,0.0,225.0,...,67.0,0.0,225.0,90.0,60.0,1799.0,Yes,3,3,2015
587184,2015-03-04,0.0,0.0,0.0,0.0,0.0,0.0,72.0,0.0,240.0,...,72.0,0.0,240.0,96.0,64.0,1920.0,Yes,4,3,2015
587185,2015-03-14,0.0,0.0,0.0,0.0,0.0,0.0,79.0,0.0,265.0,...,79.0,0.0,265.0,106.0,71.0,2118.0,Yes,14,3,2015


In [12]:
X = df.drop(['Date','Run Decision'], axis=1)
y = df['Run Decision']

In [13]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate':[0.03,0.05,0.07,],
        'n_estimators':[400,500,600,700],
        'max_delta_step' :[0,1],
        'colsample_bynode' :[0,0.5]
        }

In [14]:
xgb_model = XGBClassifier()

In [15]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [16]:
random_search=RandomizedSearchCV(xgb_model,param_distributions=params,n_iter=10,scoring='roc_auc',n_jobs=10,cv=10,verbose=True)

In [ ]:
# hyperperameter tunning
from datetime import datetime
# Here we go
start_time = timer(None)  # timing starts from this point for "start_time" variable
random_search.fit(X,y) 
timer(start_time)         # timing ends here for "start_time" variable

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 48.2min


In [1]:
random_search.best_params_

NameError: name 'random_search' is not defined

In [55]:
xgb_model = XGBClassifier(subsample= 0.6,
                          n_estimators= 400,
                          min_child_weight= 1,
                          max_depth= 4,
                          max_delta_step= 0,
                          learning_rate= 0.07,
                          gamma= 5,
                          colsample_bytree= 1.0,
                          colsample_bynode= 0.5,
                          use_label_encoder = True)

In [56]:
xgb_model.fit(X,y)

[17:12:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=0.5, colsample_bytree=1.0,
              enable_categorical=False, gamma=1, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_delta_step=1, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [57]:
from sklearn.metrics import accuracy_score

In [58]:
y_pred = xgb_model.predict(X)

In [59]:
accuracy_score(y,y_pred)

1.0